In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
!pip install torchvision

  Using cached torchvision-0.15.2-cp310-cp310-win_amd64.whl (1.2 MB)
  Using cached torch-2.0.1-cp310-cp310-win_amd64.whl (172.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1


In [2]:

# Define the transforms for data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])



In [3]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


In [7]:

# Define the CNN architectures

# Model 1
class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(16 * 16 * 16, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 16 * 16 * 16)
        x = self.fc(x)
        return x

model1 = Model1()




In [8]:

# Model 2
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(32 * 16 * 16, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 16 * 16)
        x = self.fc(x)
        return x

model2 = Model2()


In [9]:
# Model 3
class Model3(nn.Module):
    def __init__(self):
        super(Model3, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32 * 8 * 8, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = x.view(-1, 32 * 8 * 8)
        x = self.fc(x)
        return x

model3 = Model3()


In [10]:

# Model 4
class Model4(nn.Module):
    def __init__(self):
        super(Model4, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(64 * 32 * 8, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 32 * 8)
        x = self.fc(x)
        return x

model4 = Model4()

In [16]:

# Model 5
class Model5(nn.Module):
    def __init__(self):
        super(Model5, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 16 * 16)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model5 = Model5()


In [12]:
# Define the training parameters
criterion = nn.CrossEntropyLoss()
# Define the models
model1 = Model1()
model2 = Model2()
model3 = Model3()
model4 = Model4()
model5 = Model5()

# Define the optimizer for each model
optimizer1 = optim.SGD(model1.parameters(), lr=0.001, momentum=0.9)
optimizer2 = optim.SGD(model2.parameters(), lr=0.001, momentum=0.9)
optimizer3 = optim.SGD(model3.parameters(), lr=0.001, momentum=0.9)
optimizer4 = optim.SGD(model4.parameters(), lr=0.001, momentum=0.9)
optimizer5 = optim.SGD(model5.parameters(), lr=0.001, momentum=0.9)



In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1.to(device)
optimizer = torch.optim.Adam(model1.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model1(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate running loss
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print epoch statistics
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(trainloader):.4f}, Accuracy: {100 * correct / total:.2f}%")

    # Adjust learning rate
    scheduler.step()

Epoch [1/10], Loss: 1.5264, Accuracy: 46.66%
Epoch [2/10], Loss: 1.2512, Accuracy: 56.61%
Epoch [3/10], Loss: 1.1658, Accuracy: 59.62%
Epoch [4/10], Loss: 1.0981, Accuracy: 62.35%
Epoch [5/10], Loss: 1.0486, Accuracy: 63.96%
Epoch [6/10], Loss: 0.9737, Accuracy: 67.03%
Epoch [7/10], Loss: 0.9658, Accuracy: 67.46%
Epoch [8/10], Loss: 0.9601, Accuracy: 67.70%
Epoch [9/10], Loss: 0.9550, Accuracy: 67.89%
Epoch [10/10], Loss: 0.9506, Accuracy: 67.99%


In [16]:
print('Model 2 Training')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2.to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model2(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate running loss
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print epoch statistics
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(trainloader):.4f}, Accuracy: {100 * correct / total:.2f}%")

    # Adjust learning rate
    scheduler.step()

Model 2 Training
Epoch [1/10], Loss: 1.4647, Accuracy: 48.58%
Epoch [2/10], Loss: 1.1879, Accuracy: 58.62%
Epoch [3/10], Loss: 1.0796, Accuracy: 62.41%
Epoch [4/10], Loss: 1.0069, Accuracy: 65.37%
Epoch [5/10], Loss: 0.9574, Accuracy: 66.89%
Epoch [6/10], Loss: 0.8568, Accuracy: 71.20%
Epoch [7/10], Loss: 0.8450, Accuracy: 71.76%
Epoch [8/10], Loss: 0.8394, Accuracy: 71.84%
Epoch [9/10], Loss: 0.8329, Accuracy: 72.08%
Epoch [10/10], Loss: 0.8270, Accuracy: 72.20%


In [27]:
print('Model 3 Training')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model3.to(device)
optimizer = torch.optim.Adam(model3.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model3(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate running loss
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print epoch statistics
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(trainloader):.4f}, Accuracy: {100 * correct / total:.2f}%")

    # Adjust learning rate
    scheduler.step()

Model 3 Training
Epoch [1/10], Loss: 2.4672, Accuracy: 57.17%
Epoch [2/10], Loss: 2.1140, Accuracy: 63.55%
Epoch [3/10], Loss: 1.9284, Accuracy: 66.86%
Epoch [4/10], Loss: 1.8279, Accuracy: 68.70%
Epoch [5/10], Loss: 1.7324, Accuracy: 70.18%
Epoch [6/10], Loss: 1.5076, Accuracy: 74.63%
Epoch [7/10], Loss: 1.4823, Accuracy: 75.07%
Epoch [8/10], Loss: 1.4664, Accuracy: 75.52%
Epoch [9/10], Loss: 1.4545, Accuracy: 75.56%
Epoch [10/10], Loss: 1.4433, Accuracy: 75.84%


In [14]:
print('Model 4 Training')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model4.to(device)
optimizer = torch.optim.Adam(model4.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1, )

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        
        # Forward pass
        outputs = model4(inputs)
        loss = criterion(outputs, labels)
        

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate running loss
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print epoch statistics
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(trainloader):.4f}, Accuracy: {100 * correct / total:.2f}%")

    # Adjust learning rate
    scheduler.step()

Model 4 Training
Epoch [1/10], Loss: 1.3368, Accuracy: 52.79%
Epoch [2/10], Loss: 0.9591, Accuracy: 66.85%
Epoch [3/10], Loss: 0.8278, Accuracy: 71.63%
Epoch [4/10], Loss: 0.7243, Accuracy: 75.24%
Epoch [5/10], Loss: 0.6387, Accuracy: 78.09%
Epoch [6/10], Loss: 0.4761, Accuracy: 84.55%
Epoch [7/10], Loss: 0.4484, Accuracy: 85.41%
Epoch [8/10], Loss: 0.4313, Accuracy: 86.07%
Epoch [9/10], Loss: 0.4172, Accuracy: 86.64%
Epoch [10/10], Loss: 0.4035, Accuracy: 87.13%


In [ ]:
print('Model 5 Training')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model5.to(device)
optimizer = torch.optim.Adam(model5.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1, )

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        
        # Forward pass
        outputs = model5(inputs)
        loss = criterion(outputs, labels)
        

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate running loss
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print epoch statistics
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(trainloader):.4f}, Accuracy: {100 * correct / total:.2f}%")

    # Adjust learning rate
    scheduler.step()

Model 5 Training
Epoch [1/10], Loss: 1.3172, Accuracy: 52.92%
Epoch [2/10], Loss: 0.9244, Accuracy: 67.53%
Epoch [3/10], Loss: 0.7495, Accuracy: 73.72%
Epoch [4/10], Loss: 0.6017, Accuracy: 78.93%
Epoch [5/10], Loss: 0.4492, Accuracy: 84.50%
Epoch [6/10], Loss: 0.2350, Accuracy: 93.09%
Epoch [7/10], Loss: 0.1948, Accuracy: 94.50%
Epoch [8/10], Loss: 0.1696, Accuracy: 95.52%
